In [1]:
import numpy as np
import pandas as pd
import re
import nltk 
from nltk.corpus import stopwords
import pickle
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
import textdescriptives as td
import spacy
nltk.download('all')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /home/danielcrovo/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/danielcrovo/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /home/danielcrovo/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /home/danielcrovo/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /home/danielcrovo/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    |

True

In [2]:
file_path = "./age_tweets_v2 - age_tweets_v2.csv.csv"
df = pd.read_csv(file_path, sep=',')
df


,text,age_range
0,RT @forotw: Respetemos la Semana Santa. @Raulk...,35-49
1,RT @bluepoltergeist: No importa que usted sea ...,35-49
2,RT @LuxuryColumnist: What do you think of this...,35-49
3,Un milagro! Dios rtsp://v2.cache6.c.youtube.co...,35-49
4,RT @Bogota: En #LaCiudadDeLosSuenos 200 jovene...,35-49
...,...,...
300016,@CAmanecioCali uffff ! Si lagente se diera cue...,35-49
300017,Presentaran ante tribunales a camarografo dete...,35-49
300018,Me ha gustado un video de @YouTube (http://t.c...,35-49
300019,"RT @24HorasTVN: Los memes"" que identifican a l...",35-49


In [3]:
#Se revisan valores nulos
df['age_range'].isna().value_counts() 

age_range
False    299993
True         28
Name: count, dtype: int64

In [4]:
df.dropna(inplace=True)

In [5]:
df.isna().value_counts()

text   age_range
False  False        299833
Name: count, dtype: int64

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df['age_range'].unique())

LabelEncoder()

In [7]:
le.classes_

array(['13-17', '18-24', '25-34', '35-49', '50-64', '65-xx'], dtype=object)

In [8]:
df_tweets = df.copy()

In [9]:
df_tweets['classes'] = le.transform(df['age_range'])

## Preprocesamiento de Tweets

In [10]:
import re 

stop_words = set(stopwords.words('spanish'))
def remove_retweet(text):
    return re.sub('RT @[\w_]+:', '', text)

def remove_video(text):
    return re.sub('VIDEO:', '', text)

def remove_hyperlink(text):
    return re.sub(r'http\S+', '', text)

def remove_user(text):
    return re.sub('@[A-Za-z0-9_]+(:)?', '', text)

def remove_line_break(text):
    return re.sub(r'\n', '', text)

def remove_extra_spaces(text):
    return re.sub(r"\s+", " ", text)

def data_clenaning(text): 
    text = remove_retweet(text)
    text = remove_video(text)
    text = remove_hyperlink(text)
    text = remove_user(text)
    text = remove_line_break(text)
    text = remove_extra_spaces(text)
    text = re.sub(r'rtsp://[^\s]+', '',text)
    return text



In [11]:
df_tweets['processed_text'] = df_tweets['text'].apply(data_clenaning)
docs_un = df_tweets['text'].tolist()
docs_proc = df_tweets['processed_text'].tolist()

In [35]:
import spacy
# metrics = td.extract_metrics(
#     text=df_tweets['text'], 
#     spacy_model="es_core_news_lg",
#     metrics=["readability", "dependency_distance"],
# )

nlp = spacy.load("es_core_news_lg")
nlp.add_pipe("textdescriptives/descriptive_stats")

docs = nlp.pipe(docs_proc)
df_test = td.extract_df(docs)


In [38]:
df_features = df_test.drop(['text'], axis=1).join(df_tweets)

In [39]:
df_features

,dependency_distance_mean,dependency_distance_std,prop_adjacent_dependency_relation_mean,prop_adjacent_dependency_relation_std,flesch_reading_ease,flesch_kincaid_grade,smog,gunning_fog,automated_readability_index,coleman_liau_index,...,syllables_per_token_std,n_tokens,n_unique_tokens,proportion_unique_tokens,n_characters,n_sentences,text,age_range,classes,processed_text
0,0.800000,0.800000,0.200000,0.200000,18.177500,11.345000,NaN,17.000000,2.428000,0.584000,...,1.166190,5,5,1.000000,24,2,RT @forotw: Respetemos la Semana Santa. @Raulk...,35-49,3.0,Respetemos la Semana Santa.
1,1.860000,1.860000,0.160000,0.160000,46.605000,9.740000,NaN,13.133333,3.213750,5.008333,...,1.010363,24,21,0.875000,96,2,RT @bluepoltergeist: No importa que usted sea ...,35-49,3.0,"No importa que usted sea uribista, si no ve q..."
2,2.676471,2.676471,0.147059,0.147059,58.222500,7.001667,NaN,8.333333,3.358000,6.517333,...,1.299573,15,15,1.000000,69,2,RT @LuxuryColumnist: What do you think of this...,35-49,3.0,What do you think of this street art by local...
3,0.333333,0.333333,0.333333,0.333333,64.312500,4.661667,NaN,13.933333,-0.270000,-10.053333,...,0.942809,3,3,1.000000,14,2,Un milagro! Dios rtsp://v2.cache6.c.youtube.co...,35-49,3.0,Un milagro! Dios
4,1.854167,1.854167,0.208333,0.208333,25.962500,12.495000,NaN,15.034783,5.207826,7.702609,...,1.693979,23,20,0.869565,105,2,RT @Bogota: En #LaCiudadDeLosSuenos 200 jovene...,35-49,3.0,En #LaCiudadDeLosSuenos 200 jovenes ya disfru...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299828,0.900000,0.941630,0.300000,0.216025,61.742157,6.055294,8.841846,9.325490,1.628627,4.225882,...,0.903664,17,17,1.000000,73,3,"Ahora, la carne procesada o deliciosos embutid...",35-49,3.0,"Ahora, la carne procesada o deliciosos embutid..."
299829,2.666667,0.000000,0.466667,0.000000,-6.789286,17.684286,NaN,19.885714,11.475000,14.425714,...,1.444553,14,14,1.000000,78,1,RT @AlvaroAlvaradoC: Somos la burla de todo el...,35-49,3.0,Somos la burla de todo el mundo. Sacyr quiere...
299830,2.823529,0.000000,0.411765,0.000000,41.851818,10.154545,NaN,15.309091,5.050909,7.701818,...,1.028519,11,10,0.909091,55,1,"Amor, incesto y embarazo: la polemica historia...",35-49,3.0,"Amor, incesto y embarazo: la polemica historia..."
299831,1.218750,1.218750,0.187500,0.187500,69.502500,5.428333,NaN,8.333333,0.846000,3.381333,...,0.884433,15,13,0.866667,60,2,RT @camaroons: #F1 Asi quedo la #FP1 al #Malay...,35-49,3.0,#F1 Asi quedo la #FP1 al #MalaysiaGP con Rosb...


In [40]:
def tokenizer(text):
    tokenizer= TweetTokenizer()
    return tokenizer.tokenize(text)


df_tweets['tokens'] = df_tweets['processed_text'].apply(tokenizer)
    

In [41]:
df_tweets

,text,age_range,classes,processed_text,tokens
0,RT @forotw: Respetemos la Semana Santa. @Raulk...,35-49,3,Respetemos la Semana Santa.,"[Respetemos, la, Semana, Santa, .]"
1,RT @bluepoltergeist: No importa que usted sea ...,35-49,3,"No importa que usted sea uribista, si no ve q...","[No, importa, que, usted, sea, uribista, ,, si..."
2,RT @LuxuryColumnist: What do you think of this...,35-49,3,What do you think of this street art by local...,"[What, do, you, think, of, this, street, art, ..."
3,Un milagro! Dios rtsp://v2.cache6.c.youtube.co...,35-49,3,Un milagro! Dios,"[Un, milagro, !, Dios]"
4,RT @Bogota: En #LaCiudadDeLosSuenos 200 jovene...,35-49,3,En #LaCiudadDeLosSuenos 200 jovenes ya disfru...,"[En, #LaCiudadDeLosSuenos, 200, jovenes, ya, d..."
...,...,...,...,...,...
300016,@CAmanecioCali uffff ! Si lagente se diera cue...,35-49,3,uffff ! Si lagente se diera cuenta de que el ...,"[uffff, !, Si, lagente, se, diera, cuenta, de,..."
300017,Presentaran ante tribunales a camarografo dete...,35-49,3,Presentaran ante tribunales a camarografo dete...,"[Presentaran, ante, tribunales, a, camarografo..."
300018,Me ha gustado un video de @YouTube (http://t.c...,35-49,3,Me ha gustado un video de ( - Testimonio de Al...,"[Me, ha, gustado, un, video, de, (, -, Testimo..."
300019,"RT @24HorasTVN: Los memes"" que identifican a l...",35-49,3,"Los memes"" que identifican a los solteros a p...","[Los, memes, "", que, identifican, a, los, solt..."


## Extracción de caraterísticas

In [43]:
import emoji

def count_mentions(text):
    return len(re.findall(r'@[\w_]+', text))

def count_hashtags(text):
    return len(re.findall(r'#\w+', text))

def count_urls(text):
    return len(re.findall(r'http\S+', text))

def count_videos(text):
    return len(re.findall(r'rtsp://[^\s]+', text))

def count_emoticons(tokens):
    # Define a pattern to match common emoticons
    emoticon_pattern = r'(:-?\)|:-?D|;-?\)|:-?\(|:-?\[|:-?P|:\'\(|>:\(|:\||:O|:-\||:D|B\)|:o\)|:-\*|xD|X-?D|<3)'

    # Join the tokens into a string
    text = ' '.join(tokens)

    # Count the number of emoticons
    return len(re.findall(emoticon_pattern, text))

def count_uppercase(text):
    # Count the number of uppercase letters (excluding URLs)
    text_without_urls = re.sub(r'http\S+', '', text)  # Remove URLs
    return sum(1 for char in text_without_urls if char.isupper())




df_tweets['n_mentions'] = df_tweets['text'].apply(count_mentions)
df_tweets['n_hashtags'] = df_tweets['text'].apply(count_hashtags)
df_tweets['n_urls'] = df_tweets['text'].apply(count_urls)
df_tweets['n_emoticons'] = df_tweets['tokens'].apply(count_emoticons)
df_tweets['n_uppercase'] = df_tweets['text'].apply(count_uppercase)

In [44]:
df_tweets



,text,age_range,classes,processed_text,tokens,n_mentions,n_hashtags,n_urls,n_emoticons,n_uppercase
0,RT @forotw: Respetemos la Semana Santa. @Raulk...,35-49,3,Respetemos la Semana Santa.,"[Respetemos, la, Semana, Santa, .]",5,0,1,0,6
1,RT @bluepoltergeist: No importa que usted sea ...,35-49,3,"No importa que usted sea uribista, si no ve q...","[No, importa, que, usted, sea, uribista, ,, si...",1,0,0,0,4
2,RT @LuxuryColumnist: What do you think of this...,35-49,3,What do you think of this street art by local...,"[What, do, you, think, of, this, street, art, ...",2,3,1,0,7
3,Un milagro! Dios rtsp://v2.cache6.c.youtube.co...,35-49,3,Un milagro! Dios,"[Un, milagro, !, Dios]",0,0,0,0,44
4,RT @Bogota: En #LaCiudadDeLosSuenos 200 jovene...,35-49,3,En #LaCiudadDeLosSuenos 200 jovenes ya disfru...,"[En, #LaCiudadDeLosSuenos, 200, jovenes, ya, d...",1,1,0,0,12
...,...,...,...,...,...,...,...,...,...,...
300016,@CAmanecioCali uffff ! Si lagente se diera cue...,35-49,3,uffff ! Si lagente se diera cuenta de que el ...,"[uffff, !, Si, lagente, se, diera, cuenta, de,...",1,0,0,0,4
300017,Presentaran ante tribunales a camarografo dete...,35-49,3,Presentaran ante tribunales a camarografo dete...,"[Presentaran, ante, tribunales, a, camarografo...",0,1,2,0,4
300018,Me ha gustado un video de @YouTube (http://t.c...,35-49,3,Me ha gustado un video de ( - Testimonio de Al...,"[Me, ha, gustado, un, video, de, (, -, Testimo...",1,0,1,0,7
300019,"RT @24HorasTVN: Los memes"" que identifican a l...",35-49,3,"Los memes"" que identifican a los solteros a p...","[Los, memes, "", que, identifican, a, los, solt...",1,0,2,0,7


In [45]:
df_features = df_test.drop(['text'], axis=1).join(df_tweets)
df_features

,dependency_distance_mean,dependency_distance_std,prop_adjacent_dependency_relation_mean,prop_adjacent_dependency_relation_std,flesch_reading_ease,flesch_kincaid_grade,smog,gunning_fog,automated_readability_index,coleman_liau_index,...,text,age_range,classes,processed_text,tokens,n_mentions,n_hashtags,n_urls,n_emoticons,n_uppercase
0,0.800000,0.800000,0.200000,0.200000,18.177500,11.345000,NaN,17.000000,2.428000,0.584000,...,RT @forotw: Respetemos la Semana Santa. @Raulk...,35-49,3.0,Respetemos la Semana Santa.,"[Respetemos, la, Semana, Santa, .]",5.0,0.0,1.0,0.0,6.0
1,1.860000,1.860000,0.160000,0.160000,46.605000,9.740000,NaN,13.133333,3.213750,5.008333,...,RT @bluepoltergeist: No importa que usted sea ...,35-49,3.0,"No importa que usted sea uribista, si no ve q...","[No, importa, que, usted, sea, uribista, ,, si...",1.0,0.0,0.0,0.0,4.0
2,2.676471,2.676471,0.147059,0.147059,58.222500,7.001667,NaN,8.333333,3.358000,6.517333,...,RT @LuxuryColumnist: What do you think of this...,35-49,3.0,What do you think of this street art by local...,"[What, do, you, think, of, this, street, art, ...",2.0,3.0,1.0,0.0,7.0
3,0.333333,0.333333,0.333333,0.333333,64.312500,4.661667,NaN,13.933333,-0.270000,-10.053333,...,Un milagro! Dios rtsp://v2.cache6.c.youtube.co...,35-49,3.0,Un milagro! Dios,"[Un, milagro, !, Dios]",0.0,0.0,0.0,0.0,44.0
4,1.854167,1.854167,0.208333,0.208333,25.962500,12.495000,NaN,15.034783,5.207826,7.702609,...,RT @Bogota: En #LaCiudadDeLosSuenos 200 jovene...,35-49,3.0,En #LaCiudadDeLosSuenos 200 jovenes ya disfru...,"[En, #LaCiudadDeLosSuenos, 200, jovenes, ya, d...",1.0,1.0,0.0,0.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299828,0.900000,0.941630,0.300000,0.216025,61.742157,6.055294,8.841846,9.325490,1.628627,4.225882,...,"Ahora, la carne procesada o deliciosos embutid...",35-49,3.0,"Ahora, la carne procesada o deliciosos embutid...","[Ahora, ,, la, carne, procesada, o, deliciosos...",0.0,0.0,0.0,0.0,3.0
299829,2.666667,0.000000,0.466667,0.000000,-6.789286,17.684286,NaN,19.885714,11.475000,14.425714,...,RT @AlvaroAlvaradoC: Somos la burla de todo el...,35-49,3.0,Somos la burla de todo el mundo. Sacyr quiere...,"[Somos, la, burla, de, todo, el, mundo, ., Sac...",1.0,0.0,0.0,0.0,7.0
299830,2.823529,0.000000,0.411765,0.000000,41.851818,10.154545,NaN,15.309091,5.050909,7.701818,...,"Amor, incesto y embarazo: la polemica historia...",35-49,3.0,"Amor, incesto y embarazo: la polemica historia...","[Amor, ,, incesto, y, embarazo, :, la, polemic...",0.0,0.0,2.0,0.0,2.0
299831,1.218750,1.218750,0.187500,0.187500,69.502500,5.428333,NaN,8.333333,0.846000,3.381333,...,RT @camaroons: #F1 Asi quedo la #FP1 al #Malay...,35-49,3.0,#F1 Asi quedo la #FP1 al #MalaysiaGP con Rosb...,"[#F1, Asi, quedo, la, #FP1, al, #MalaysiaGP, c...",1.0,3.0,1.0,0.0,13.0


In [56]:
from sklearn.model_selection import train_test_split
features = df_features.drop(['text', 'age_range', 'processed_text', 'tokens', 'smog'], axis=1).dropna()
features.to_csv('./features.csv')
X = features.values
y = features['classes'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Probando un classificador sencillo (baseline)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score


pipe = Pipeline([('scaler', StandardScaler())])


In [126]:
modelB = LogisticRegression ()

X_train_s = X_train
X_test_s = X_test
modelB.fit(X_train_s, y_train)
train_preds = modelB.predict(X_train_s)
preds = modelB.predict(X_test_s)
train_f1 = f1_score(y_train, train_preds, average='micro')
test_f1 = f1_score(y_test, preds, average='micro')
train_acc = accuracy_score(y_train, train_preds)
test_acc = accuracy_score(y_test, preds)
train_roc = roc_auc_score(y_train, modelB.predict_proba(X_train_s), multi_class='ovr')
test_roc = roc_auc_score(y_test, modelB.predict_proba(X_test_s), multi_class='ovr')

print(f'Model: Logistic Regression')
print(f'Training F1: {train_f1}')
print(f'Test F1: {test_f1}')
print(f'Training ROC AUC: {train_roc}')
print(f'Test ROC AUC: {test_roc}')
print(f'Training Acc: {train_acc}')
print(f'Test Acc: {test_acc}')
print('-----------------------')   

/home/danielcrovo/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model: Logistic Regression
Training F1: 0.9225541986991358
Test F1: 0.9206983709873175
Training ROC AUC: 0.9894945608653657
Test ROC AUC: 0.9891175685638739
Training Acc: 0.9225541986991358
Test Acc: 0.9206983709873175
-----------------------


In [127]:
import seaborn as sns
import matplotlib.pyplot as plt
from timeit import timeit
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier




In [128]:
def get_models():
    models = dict()
    
    models['Logistic Regression'] = LogisticRegression(random_state=42, max_iter=1000)
    
    models['Decision Tree'] = DecisionTreeClassifier(random_state=42)
    
    models['Random Forest'] = RandomForestClassifier(random_state=42)
    
    models['Extra Trees'] = ExtraTreesClassifier(random_state=42)
    
    models['Gradient Boosting'] = GradientBoostingClassifier(random_state=42)
    
    models['Hist Gradient Boosting'] = HistGradientBoostingClassifier(random_state=42)
    
    models['AdaBoost'] = AdaBoostClassifier(random_state=42) 
                
    
    return models

In [129]:
models = get_models()
results = list()
def evaluate_models(X_train, X_test, y_train, y_test, pipe):
    for name, model in models.items():
        X_train_s = pipe.fit_transform(X_train)
        X_test_s = pipe.fit_transform(X_test)
        model.fit(X_train_s, y_train)
        train_preds = model.predict(X_train_s)
        preds = model.predict(X_test_s)
        train_f1 = f1_score(y_train, train_preds, average='micro')
        test_f1 = f1_score(y_test, preds, average='micro')
        train_acc = accuracy_score(y_train, train_preds)
        test_acc = accuracy_score(y_test, preds)
        train_roc = roc_auc_score(y_train, model.predict_proba(X_train_s), multi_class='ovr')
        test_roc = roc_auc_score(y_test, model.predict_proba(X_test_s), multi_class='ovr')
        scores = {'name': name, 
                  'train_f1': train_f1, 
                  'test_f1': test_f1, 
                  'train_roc': train_roc, 
                  'test_roc': test_roc,
                  'train_acc': train_acc, 
                  'test_acc': test_acc}
        
        
        print(f'Model: {name}')
        print(f'Training F1: {train_f1}')
        print(f'Test F1: {test_f1}')
        print(f'Training ROC AUC: {train_roc}')
        print(f'Test ROC AUC: {test_roc}')
        print(f'Training Acc: {train_acc}')
        print(f'Test Acc: {test_acc}')
        print('-----------------------')        
        results.append(scores)



In [130]:
evaluate_models(X_train, X_test, y_train, y_test, pipe)

Model: Logistic Regression
Training F1: 1.0
Test F1: 1.0
Training ROC AUC: 1.0
Test ROC AUC: 1.0
Training Acc: 1.0
Test Acc: 1.0
-----------------------
Model: Decision Tree
Training F1: 1.0
Test F1: 1.0
Training ROC AUC: 1.0
Test ROC AUC: 1.0
Training Acc: 1.0
Test Acc: 1.0
-----------------------
Model: Random Forest
Training F1: 1.0
Test F1: 0.9997995746528744
Training ROC AUC: 1.0
Test ROC AUC: 0.999999727815973
Training Acc: 1.0
Test Acc: 0.9997995746528744
-----------------------
Model: Extra Trees
Training F1: 1.0
Test F1: 1.0
Training ROC AUC: 1.0
Test ROC AUC: 1.0
Training Acc: 1.0
Test Acc: 1.0
-----------------------


# Punto 2
